# API for SimpleFIX : Algorithmic Code Generation using QuickFIX's XML File.

The code generated below is an API for SimpleFix for FIX version 4.4.

SimpleFIX : https://simplefix.readthedocs.io/en/latest

SimpleFIX GitHub: https://github.com/da4089/simplefix

QuickFIX: http://www.quickfixengine.org/

We will create a function for each message type, with all required tags as compulsory arguments to the function, and all non-essential tags as default arguments which will be 'None' by default.

For Components in FIX, we define classes for each component inheriting from a base Component Class, each of which need to have a tag_vals attribute. All the tags and corresponding values will be stored in this when the component is initialised. These components have to be provided to the message as arguments(an instance of the corresponding component class).

```python
class Component:
    def __init__(self):
        self.tag_vals = []
```

As before, all required tags are compulsory arguments to the constructor function (when the object is created), and all non-essential tags as default arguments which will be 'None' by default.

For Groups, we use a special structure which is as follows:
```python
group_name = [[Tag_1_Name, Tag_2_Name, ... Tag_M_Name],
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M], #For group instance 1
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M], #For group instance 2
              .
              .
              .
              [Value_Tag_1, Value_Tag_2, ... Value_Tag_M]] #For group instance N
```

Thus, the number of instances of the group is 
```python
len(group_name)-1
```

For parsing a received message, we create a function parse_message(), which takes as input an instance of the SimpleFIX FixMessage, which is obtained after calling the FixParser.get_message(). This is decoded to provide names of tags instead of mere numbers, and returned in a structure as follows:

```python
msg_dict = {Tag_Name_1:Val_1,
            Tag_Name_2:Val_1,
            .
            .
            .
            Tag_Name_N:Val_N}
```

Groups are returned in the same structure as before, thus allowing convenient usage of these.

## Code Generation

### xmltodict will read the xml file and convert to a dictionary so we can expolit the structure of the XML File.

In [1]:
import xmltodict

In [2]:
#Open the file, and create the dictionary.
file = open('FIX44.xml', mode = 'rb')
data = xmltodict.parse(file)

In [3]:
groups_to_fields = {} #Will store the mappings of GroupName to Fields and Components

In [4]:
header_required_tags = []
x=data['fix']['header']
for field in x['field']:
    if field['@required'] == 'N':
        header_required_tags.append(field['@name'])
groups_to_fields[x['group']['@name']] = [i['@name'] for i in x['group']['field']]
header_required_tags.append(x['group']['@name'])

In [5]:
tag_num_to_name = [None]*957 #Will store the mapping of tag name to tag number
tag_name_to_num = {} #Will store the reverse mapping
tag_id_desc_to_num = [None]*957 #Will store the mappings of values allowed if type is enum

In [6]:
for x in data['fix']['fields']['field']:
    tag_num_to_name[int(x['@number'])] = x['@name']
    tag_name_to_num[x['@name']] = int(x['@number'])
    if 'value' in x.keys():
        tag_id_desc_to_num.insert(int(x['@number']),{k['@description']:k['@enum'] for k in x['value']})

In [7]:
components_names = [i['@name'] for i in data['fix']['components']['component']] #Will store the names of the components

In [8]:
component_class_templates = [['class Component:', '\tdef __init__(self):','\t\tself.tag_vals = []']] #Will store the Component Classes Code

In [9]:
for x in data['fix']['components']['component']:
    components_classes = [] #Code for current Component Class
    c_name = x['@name'] #Name of the Component
    components_classes.append(f'class {c_name}(Component):')
    reqd = [] #Required Tags
    not_reqd = [] #Non-Required Tags
    
    if 'field' in x.keys():
        fields = x['field']
        if type(fields)==list:
            for arg in fields:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if fields['@required'] == 'Y':
                reqd.append(f"{fields['@name']}")
            else:
                not_reqd.append(f"{fields['@name']}=None")
                                
    if 'component' in x.keys():
        components = x['component']
        if type(components)==list:
            for arg in components:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if components['@required'] == 'Y':
                reqd.append(f"{components['@name']}")
            else:
                not_reqd.append(f"{components['@name']}=None")
                                
    if 'group' in x.keys():
        groups_m = x['group']
        if type(groups_m)==list:
            for arg in groups_m:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
                #Groups contain other fields, so that information must be stored.
                if 'field' in arg.keys():
                    if type(groups_m['field']) == list:
                        groups_to_fields[arg['@name']] = [i['@name'] for i in arg['field']]
                    else:
                        groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
                #Groups contain other components, so that information must be stored.
                if 'component' in arg.keys():
                    if type(groups_m['component']) == list:
                        groups_to_fields[arg['@name']] += [i['@name'] for i in arg['component']]
                    else:
                        groups_to_fields[groups_m['@name']] += [groups_m['component']['@name']]
        else:
            if groups_m['@required'] == 'Y':
                reqd.append(f"{groups_m['@name']}")
            else:
                not_reqd.append(f"{groups_m['@name']}=None")
            
            if 'field' in groups_m.keys():
                if type(groups_m['field'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['field']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
            if 'component' in groups_m.keys():
                if type(groups_m['component'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['component']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['component']['@name']]
                                
    components_classes.append(f"\tdef __init__(self, {(','.join(reqd+not_reqd))}):")
    components_classes.append('\t\tself.tag_vals = []')
                                
    for tag in reqd: #Iterate through each compulsory tag
        if tag not in components_names: 
            if tag not in groups_to_fields.keys():
                # It's a tag
                components_classes.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},{tag}))')
            else:
                # It's a group
                components_classes.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                components_classes.append(f'\t\tfor i in range(1,len({tag})):')
                components_classes.append(f'\t\t\tfor idx,attr in enumerate({tag}[0]):')
                components_classes.append(f'\t\t\t\tif attr in components_names:')
                components_classes.append(f'\t\t\t\t\tself.tag_vals+=attr.tag_vals')
                components_classes.append(f'\t\t\t\telse:')
                components_classes.append(f'\t\t\t\t\tself.tag_vals.append(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            #It's a component
            components_classes.append(f'\t\tself.tag_vals+={tag}.tag_vals')
    for tag in not_reqd:
        tag = tag[:-5] #To remove the "=None" at the end of non-essential tags that we added previously
        components_classes.append(f'\t\tif {tag} != None:')
                                
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                #It's a tag
                components_classes.append(f'\t\t\tself.tag_vals.append(({tag_name_to_num[tag]},{tag}))')
            else:
                #It's a group
                components_classes.append(f'\t\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                components_classes.append(f'\t\t\tfor i in range(1,len({tag})):')
                components_classes.append(f'\t\t\t\tfor idx,attr in enumerate({tag}[0]):')
                components_classes.append(f'\t\t\t\t\tif attr in components_names:')
                components_classes.append(f'\t\t\t\t\t\tself.tag_vals+=attr.tag_vals')
                components_classes.append(f'\t\t\t\t\telse:')
                components_classes.append(f'\t\t\t\t\t\tself.tag_vals.append(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            #It's a Component
             components_classes.append(f'\t\t\tself.tag_vals+={tag}.tag_vals')
    component_class_templates.append(components_classes)

In [10]:
for i in component_class_templates:
    for j in i:
        print(j)
    print('\n')

class Component:
	def __init__(self):
		self.tag_vals = []


class CommissionData(Component):
	def __init__(self, Commission=None,CommType=None,CommCurrency=None,FundRenewWaiv=None):
		self.tag_vals = []
		if Commission != None:
			self.tag_vals.append((12,Commission))
		if CommType != None:
			self.tag_vals.append((13,CommType))
		if CommCurrency != None:
			self.tag_vals.append((479,CommCurrency))
		if FundRenewWaiv != None:
			self.tag_vals.append((497,FundRenewWaiv))


class DiscretionInstructions(Component):
	def __init__(self, DiscretionInst=None,DiscretionOffsetValue=None,DiscretionMoveType=None,DiscretionOffsetType=None,DiscretionLimitType=None,DiscretionRoundDirection=None,DiscretionScope=None):
		self.tag_vals = []
		if DiscretionInst != None:
			self.tag_vals.append((388,DiscretionInst))
		if DiscretionOffsetValue != None:
			self.tag_vals.append((389,DiscretionOffsetValue))
		if DiscretionMoveType != None:
			self.tag_vals.append((841,DiscretionMoveType))
		if DiscretionOff

In [11]:
final_templates_messages = [] #Will store the code for the messages functions.

In [12]:
for msg in data['fix']['messages']['message']:
    final_templates = [] #Code of current message
    name = msg['@name'] #Name of message
    msg_type = msg['@msgtype']
    reqd = []
    not_reqd = []
    
    #For all field attributes
    if 'field' in msg.keys():
        fields = msg['field']
        if type(fields)==list:
            for arg in fields:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if fields['@required'] == 'Y':
                reqd.append(f"{fields['@name']}")
            else:
                not_reqd.append(f"{fields['@name']}=None")
                                
    #For all component type attributes
    if 'component' in msg.keys():
        components = msg['component']
        if type(components)==list:
            for arg in components:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
        else:
            if components['@required'] == 'Y':
                reqd.append(f"{components['@name']}")
            else:
                not_reqd.append(f"{components['@name']}=None")
                                
    if 'group' in msg.keys():
        groups_m = msg['group']
        if type(groups_m)==list:
            for arg in groups_m:
                if arg['@required'] == 'Y':
                    reqd.append(f"{arg['@name']}")
                else:
                    not_reqd.append(f"{arg['@name']}=None")
                if 'field' in arg.keys():
                    if type(groups_m['field']) == list:
                        groups_to_fields[arg['@name']] = [i['@name'] for i in arg['field']]
                    else:
                        groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
                if 'component' in arg.keys():
                    if type(groups_m['component']) == list:
                        groups_to_fields[arg['@name']] += [i['@name'] for i in arg['component']]
                    else:
                        groups_to_fields[groups_m['@name']] += [groups_m['component']['@name']]
        else:
            if groups_m['@required'] == 'Y':
                reqd.append(f"{groups_m['@name']}")
            else:
                not_reqd.append(f"{groups_m['@name']}=None")
            if 'field' in groups_m.keys():
                if type(groups_m['field'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['field']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['field']['@name']]
            if 'component' in groups_m.keys():
                if type(groups_m['component'])==list:
                    groups_to_fields[groups_m['@name']] = [i['@name'] for i in groups_m['component']]
                else:
                    groups_to_fields[groups_m['@name']] = [groups_m['component']['@name']]
    
    final_templates.append('def '+name+'(MsgSeqNum,TargetCompID,'+','.join(reqd+not_reqd)+'):')
    final_templates.append('\tmsg=simplefix.FixMessage()')
    final_templates.append('\tmsg.append_pair(8,"FIX.4.4",header=True)')
    final_templates.append(f'\tmsg.append_pair(35,{msg_type},header=True)')
    final_templates.append('\tmsg.append_pair(49,SENDER_COMP_ID,header=True)')
    final_templates.append('\tmsg.append_pair(34,MsgSeqNum,header=True)')
    final_templates.append('\tmsg.append_pair(56,TargetCompID,header=True)')
    for tag in reqd:
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                final_templates.append(f'\tmsg.append_pair({tag_name_to_num[tag]},{tag})')
            else:
                final_templates.append(f'\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                final_templates.append(f'\tfor i in range(1,len({tag})):')
                final_templates.append(f'\t\tfor idx,attr in enumerate({tag}[0]):')
                final_templates.append(f'\t\t\tif attr in components_names:')
                final_templates.append(f'\t\t\t\tfor tv in attr.tag_vals:')
                final_templates.append(f'\t\t\t\t\tmsg.append_pair(tv[0],tv[1])')
                final_templates.append(f'\t\t\telse:')
                final_templates.append(f'\t\t\t\tmsg.append_pair(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            final_templates.append(f'\tfor tv in {tag}.tag_vals:')
            final_templates.append(f'\t\tmsg.append_pair(tv[0],tv[1])')
    for tag in not_reqd:
        tag = tag[:-5]
        final_templates.append(f'\tif {tag} != None:')
        if tag not in components_names:
            if tag not in groups_to_fields.keys():
                final_templates.append(f'\t\tmsg.append_pair({tag_name_to_num[tag]},{tag})')
            else:
                final_templates.append(f'\t\tself.tag_vals.append(({tag_name_to_num[tag]},len({tag})-1))')
                final_templates.append(f'\t\tfor i in range(1,len({tag})):')
                final_templates.append(f'\t\t\tfor idx,attr in enumerate({tag}[0]):')
                final_templates.append(f'\t\t\t\tif attr in components_names:')
                final_templates.append(f'\t\t\t\t\tfor tv in attr.tag_vals:')
                final_templates.append(f'\t\t\t\t\t\tmsg.append_pair(tv[0],tv[1])')
                final_templates.append(f'\t\t\t\telse:')
                final_templates.append(f'\t\t\t\t\tmsg.append_pair(tag_name_to_num[attr],{tag}[i][idx])')
        else:
            final_templates.append(f'\t\tfor tv in {tag}.tag_vals:')
            final_templates.append(f'\t\t\tmsg.append_pair(tv[0],tv[1])')
    final_templates.append(f'\treturn msg.encode()')
    final_templates_messages.append(final_templates)

In [13]:
for i in final_templates_messages:
    for j in i:
        print(j)
    print('\n')

def Heartbeat(MsgSeqNum,TargetCompID,TestReqID=None):
	msg=simplefix.FixMessage()
	msg.append_pair(8,"FIX.4.4",header=True)
	msg.append_pair(35,0,header=True)
	msg.append_pair(49,SENDER_COMP_ID,header=True)
	msg.append_pair(34,MsgSeqNum,header=True)
	msg.append_pair(56,TargetCompID,header=True)
	if TestReqID != None:
		msg.append_pair(112,TestReqID)
	return msg.encode()


def TestRequest(MsgSeqNum,TargetCompID,TestReqID):
	msg=simplefix.FixMessage()
	msg.append_pair(8,"FIX.4.4",header=True)
	msg.append_pair(35,1,header=True)
	msg.append_pair(49,SENDER_COMP_ID,header=True)
	msg.append_pair(34,MsgSeqNum,header=True)
	msg.append_pair(56,TargetCompID,header=True)
	msg.append_pair(112,TestReqID)
	return msg.encode()


def ResendRequest(MsgSeqNum,TargetCompID,BeginSeqNo,EndSeqNo):
	msg=simplefix.FixMessage()
	msg.append_pair(8,"FIX.4.4",header=True)
	msg.append_pair(35,2,header=True)
	msg.append_pair(49,SENDER_COMP_ID,header=True)
	msg.append_pair(34,MsgSeqNum,header=True)
	msg.append_pair(5

		msg.append_pair(625,TradingSessionSubID)
	if UnsolicitedIndicator != None:
		msg.append_pair(325,UnsolicitedIndicator)
	if SecurityTradingStatus != None:
		msg.append_pair(326,SecurityTradingStatus)
	if FinancialStatus != None:
		msg.append_pair(291,FinancialStatus)
	if CorporateAction != None:
		msg.append_pair(292,CorporateAction)
	if HaltReasonChar != None:
		msg.append_pair(327,HaltReasonChar)
	if InViewOfCommon != None:
		msg.append_pair(328,InViewOfCommon)
	if DueToRelated != None:
		msg.append_pair(329,DueToRelated)
	if BuyVolume != None:
		msg.append_pair(330,BuyVolume)
	if SellVolume != None:
		msg.append_pair(331,SellVolume)
	if HighPx != None:
		msg.append_pair(332,HighPx)
	if LowPx != None:
		msg.append_pair(333,LowPx)
	if LastPx != None:
		msg.append_pair(31,LastPx)
	if TransactTime != None:
		msg.append_pair(60,TransactTime)
	if Adjustment != None:
		msg.append_pair(334,Adjustment)
	if Text != None:
		msg.append_pair(58,Text)
	if EncodedTextLen != None:
		msg.append_pai

		msg.append_pair(645,MktBidPx)
	if MktOfferPx != None:
		msg.append_pair(646,MktOfferPx)
	if MinBidSize != None:
		msg.append_pair(647,MinBidSize)
	if BidSize != None:
		msg.append_pair(134,BidSize)
	if MinOfferSize != None:
		msg.append_pair(648,MinOfferSize)
	if OfferSize != None:
		msg.append_pair(135,OfferSize)
	if ValidUntilTime != None:
		msg.append_pair(62,ValidUntilTime)
	if BidSpotRate != None:
		msg.append_pair(188,BidSpotRate)
	if OfferSpotRate != None:
		msg.append_pair(190,OfferSpotRate)
	if BidForwardPoints != None:
		msg.append_pair(189,BidForwardPoints)
	if OfferForwardPoints != None:
		msg.append_pair(191,OfferForwardPoints)
	if MidPx != None:
		msg.append_pair(631,MidPx)
	if BidYield != None:
		msg.append_pair(632,BidYield)
	if MidYield != None:
		msg.append_pair(633,MidYield)
	if OfferYield != None:
		msg.append_pair(634,OfferYield)
	if TransactTime != None:
		msg.append_pair(60,TransactTime)
	if OrdType != None:
		msg.append_pair(40,OrdType)
	if BidForwardPoints2 !

In [14]:
my_file = open('SimpleFIX_API_1.py','w')

In [15]:
my_file.writelines('import simplefix\n\n')
my_file.writelines('SENDER_COMP_ID = "COEP_Credit_Suisse"\n\n')
my_file.writelines(f"""components_names=[{",".join([f'"{c}"' for c in components_names])}]\n\n""")
my_file.writelines('tag_name_to_num={'+",".join([f'"{k}":{tag_name_to_num[k]}' for k in tag_name_to_num.keys()])+'}\n\n')
for f in component_class_templates:
    for j in f:
        my_file.writelines(j+'\n')
    my_file.writelines('\n')
for f in final_templates_messages:
    for j in f:
        my_file.writelines(j+'\n')
    my_file.writelines('\n')

In [16]:
my_file.close()

### Now, we generate the message parsing functions. For this we extract all tags possible in each message(including components and groups).

In [17]:
def group_tags(group): #This will return all tags in a group (including components unwrapping)
    tag_list = []
    if 'field' in group.keys():
        fields = group['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in group.keys():
        components = group['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in group.keys():
        raise ValueError()  #Since assumption of no direct recursion of groups i.e. a group within a group
    return tag_list

def component_tags(component_name): #This will return all tags in a component (including components, groups unwrapping)
    tag_list = []
    for i in data['fix']['components']['component']:
        if i['@name'] == component_name:
            component = i
            break
    if 'field' in component.keys():
        fields = component['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in component.keys():
        components = component['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in component.keys():
        groups = component['group']
        if type(groups) == list:
            for group in groups:
                tag_list.append([tag_name_to_num[group['@name']],group_tags(group)])
        else:
            tag_list.append([tag_name_to_num[groups['@name']],group_tags(groups)])
    return tag_list
                
final_tags = {} #This will store the tags corresponding to a message

for msg in data['fix']['messages']['message']:
    msg_idx = msg['@msgtype']
    tag_list = []
    if 'field' in msg.keys():
        fields = msg['field']
        if type(fields)==list:
            tag_list+=[tag_name_to_num[i['@name']] for i in fields]
        else:
            tag_list.append(tag_name_to_num[fields['@name']])
    if 'component' in msg.keys():
        components = msg['component']
        if type(components)==list:
            for comp in components:
                tag_list.append([comp['@name'],component_tags(comp['@name'])])
        else:
            tag_list.append([components['@name'],component_tags(components['@name'])])
    if 'group' in msg.keys():
        groups = msg['group']
        if type(groups) == list:
            for group in groups:
                tag_list.append([tag_name_to_num[group['@name']],group_tags(group)])
        else:
            tag_list.append([tag_name_to_num[groups['@name']],group_tags(groups)])
    final_tags[msg_idx] = tag_list

In [18]:
final_tags

{'0': [112],
 '1': [112],
 '2': [7, 16],
 '3': [45, 371, 372, 373, 58, 354, 355],
 '4': [123, 36],
 '5': [58, 354, 355],
 '6': [23,
  28,
  26,
  54,
  854,
  27,
  15,
  423,
  44,
  62,
  25,
  130,
  58,
  354,
  355,
  60,
  149,
  ['Instrument',
   [55,
    65,
    48,
    22,
    460,
    461,
    167,
    762,
    200,
    541,
    201,
    224,
    225,
    239,
    226,
    227,
    228,
    255,
    543,
    470,
    471,
    472,
    240,
    202,
    947,
    206,
    231,
    223,
    207,
    106,
    348,
    349,
    107,
    350,
    351,
    691,
    667,
    875,
    876,
    873,
    874,
    ['SecAltIDGrp', [[454, [455, 456]]]],
    ['EvntGrp', [[864, [865, 866, 867, 868]]]]]],
  ['FinancingDetails', [913, 914, 915, 918, 788, 916, 917, 919, 898]],
  ['UndInstrmtGrp',
   [[711,
     [['UnderlyingInstrument',
       [311,
        312,
        309,
        305,
        462,
        463,
        310,
        763,
        313,
        542,
        315,
        241,
    

In [19]:
header_tags = []
x=data['fix']['header']
for field in x['field']:
    header_tags.append(tag_name_to_num[field['@name']])
header_tags.append([tag_name_to_num[x['group']['@name']],group_tags(x['group'])])

In [20]:
header_tags

[8,
 9,
 35,
 49,
 56,
 115,
 128,
 90,
 91,
 34,
 50,
 142,
 57,
 143,
 116,
 144,
 129,
 145,
 43,
 97,
 52,
 122,
 212,
 213,
 347,
 369,
 [627, [628, 629, 630]]]

In [21]:
def get_msg_name(key_num): #Gets the name of the message from it's message type.
    for i in data['fix']['messages']['message']:
        if i['@msgtype'] == str(key_num):
            return i['@name']

In [22]:
def parse_component(component,num_tabs): #Generation of code when inside a component.
    for c_id in component[1]:
        if type(c_id) != list: #Field
            parse_messages_code.append('\t'*num_tabs+f'if {c_id} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[c_id]}"] = str(msg.get({c_id}),"utf-8")')
        elif type(c_id[0]) != str: #Group
            parse_messages_code.append('\t'*num_tabs+f'if {c_id[0]} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[c_id[0]]}"] = [[]]')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({c_id[0]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\t\tmsg_dict["{tag_num_to_name[c_id[0]]}"].append([])')
            parse_group(tag_num_to_name[c_id[0]],c_id[1],num_tabs+1)
        else: #Component
            parse_component(c_id, num_tabs)
            
def parse_group(tag_name, group_tags, num_tabs): #Generation of code when inside a group.
    for g_id in group_tags:
        if type(g_id) != list: #Field
            parse_messages_code.append('\t'*num_tabs+f'if {g_id} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_name}"][0].append("{tag_num_to_name[g_id]}")')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({tag_name_to_num[tag_name]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\tmsg_dict["{tag_name}"][i+1].append(str(msg.get({g_id},i+1),"utf-8"))')
        elif type(g_id[0]) != str: #Group
            parse_messages_code.append('\t'*num_tabs+f'if {tag_name_to_num[g_id[0]]} in msg:')
            parse_messages_code.append('\t'*num_tabs+f'\tmsg_dict["{tag_num_to_name[g_id[0]]}"] = [[]]')
            parse_messages_code.append('\t'*num_tabs+f'\tfor i in range(int(msg.get({g_id[0]}))):')
            parse_messages_code.append('\t'*num_tabs+f'\t\t\tmsg_dict["{tag_num_to_name[g_id[0]]}"].append([])')
            parse_group(g_id[0],g_id[1],num_tabs+1)
        else: #Component
            parse_component(g_id, num_tabs)

parse_messages_code = ['def parse_message(msg):'] #Will store the final parse_message() code
parse_messages_code.append('\tmsg_dict = {}')
parse_messages_code.append('\tmsgtype = msg.get(35)')
for h_id in header_tags:
    if type(h_id) != list:
        parse_messages_code.append(f'\tif {h_id} in msg:')
        parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[h_id]}"] = str(msg.get({h_id}),"utf-8")')
    elif type(h_id[0]) != str:
        parse_messages_code.append(f'\tif {h_id[0]} in msg:')
        parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[h_id[0]]}"] = [[]]')
        parse_messages_code.append(f'\t\tfor i in range(int(msg.get({h_id[0]}))):')
        parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[h_id[0]]}"].append([])')
        parse_group(tag_num_to_name[h_id[0]],h_id[1],2)
    
count = 0
for key in final_tags.keys():
    if count == 0:
        parse_messages_code.append(f'\tif str(msgtype,"utf-8")=="{key}":')
    elif count != 92:
        parse_messages_code.append(f'\telif str(msgtype,"utf-8")=="{key}":')
    else:
        parse_messages_code.append(f'\telse:')
    parse_messages_code.append(f'\t\tmsg_dict["{tag_num_to_name[35]}"] = "{get_msg_name(key)}"')
    for tag in final_tags[key]:
        if type(tag) != list:
            parse_messages_code.append(f'\t\tif {tag} in msg:')
            parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[tag]}"] = str(msg.get({tag}),"utf-8")')
        elif type(tag[0]) != str:
            parse_messages_code.append(f'\t\tif {tag[0]} in msg:')
            parse_messages_code.append(f'\t\t\tmsg_dict["{tag_num_to_name[tag[0]]}"] = [[]]')
            parse_messages_code.append(f'\t\t\tfor i in range(int(msg.get({tag[0]}))):')
            parse_messages_code.append(f'\t\t\t\tmsg_dict["{tag_num_to_name[tag[0]]}"].append([])')
            parse_group(tag_num_to_name[tag[0]],tag[1],3)
        else:
            parse_component(tag,2)
    count+=1
parse_messages_code.append('\treturn msg_dict')

In [23]:
my_file_parse = open('SimpleFIX_API_2.py','w')
for i in parse_messages_code:
    my_file_parse.writelines(i+'\n')
my_file_parse.close()

In [24]:
for i in parse_messages_code:
    print(i)

def parse_message(msg):
	msg_dict = {}
	msgtype = msg.get(35)
	if 8 in msg:
		msg_dict["BeginString"] = str(msg.get(8),"utf-8")
	if 9 in msg:
		msg_dict["BodyLength"] = str(msg.get(9),"utf-8")
	if 35 in msg:
		msg_dict["MsgType"] = str(msg.get(35),"utf-8")
	if 49 in msg:
		msg_dict["SenderCompID"] = str(msg.get(49),"utf-8")
	if 56 in msg:
		msg_dict["TargetCompID"] = str(msg.get(56),"utf-8")
	if 115 in msg:
		msg_dict["OnBehalfOfCompID"] = str(msg.get(115),"utf-8")
	if 128 in msg:
		msg_dict["DeliverToCompID"] = str(msg.get(128),"utf-8")
	if 90 in msg:
		msg_dict["SecureDataLen"] = str(msg.get(90),"utf-8")
	if 91 in msg:
		msg_dict["SecureData"] = str(msg.get(91),"utf-8")
	if 34 in msg:
		msg_dict["MsgSeqNum"] = str(msg.get(34),"utf-8")
	if 50 in msg:
		msg_dict["SenderSubID"] = str(msg.get(50),"utf-8")
	if 142 in msg:
		msg_dict["SenderLocationID"] = str(msg.get(142),"utf-8")
	if 57 in msg:
		msg_dict["TargetSubID"] = str(msg.get(57),"utf-8")
	if 143 in msg:
		msg_dict["TargetLocation

				msg_dict["NoUnderlyingStips"] = [[]]
				for i in range(int(msg.get(887))):
						msg_dict["NoUnderlyingStips"].append([])
				if 888 in msg:
					msg_dict["NoUnderlyingStips"][0].append("UnderlyingStipType")
					for i in range(int(msg.get(887))):
						msg_dict["NoUnderlyingStips"][i+1].append(str(msg.get(888,i+1),"utf-8"))
				if 889 in msg:
					msg_dict["NoUnderlyingStips"][0].append("UnderlyingStipValue")
					for i in range(int(msg.get(887))):
						msg_dict["NoUnderlyingStips"][i+1].append(str(msg.get(889,i+1),"utf-8"))
		if 33 in msg:
			msg_dict["NoLinesOfText"] = [[]]
			for i in range(int(msg.get(33))):
					msg_dict["NoLinesOfText"].append([])
			if 58 in msg:
				msg_dict["NoLinesOfText"][0].append("Text")
				for i in range(int(msg.get(33))):
					msg_dict["NoLinesOfText"][i+1].append(str(msg.get(58,i+1),"utf-8"))
			if 354 in msg:
				msg_dict["NoLinesOfText"][0].append("EncodedTextLen")
				for i in range(int(msg.get(33))):
					msg_dict["NoLinesOfText"][i+1].appen

				if 233 in msg:
					msg_dict["NoStipulations"][0].append("StipulationType")
					for i in range(int(msg.get(232))):
						msg_dict["NoStipulations"][i+1].append(str(msg.get(233,i+1),"utf-8"))
				if 234 in msg:
					msg_dict["NoStipulations"][0].append("StipulationValue")
					for i in range(int(msg.get(232))):
						msg_dict["NoStipulations"][i+1].append(str(msg.get(234,i+1),"utf-8"))
			if 38 in msg:
				msg_dict["OrderQty"] = str(msg.get(38),"utf-8")
			if 152 in msg:
				msg_dict["CashOrderQty"] = str(msg.get(152),"utf-8")
			if 516 in msg:
				msg_dict["OrderPercent"] = str(msg.get(516),"utf-8")
			if 468 in msg:
				msg_dict["RoundingDirection"] = str(msg.get(468),"utf-8")
			if 469 in msg:
				msg_dict["RoundingModulus"] = str(msg.get(469),"utf-8")
			if 218 in msg:
				msg_dict["Spread"] = str(msg.get(218),"utf-8")
			if 220 in msg:
				msg_dict["BenchmarkCurveCurrency"] = str(msg.get(220),"utf-8")
			if 221 in msg:
				msg_dict["BenchmarkCurveName"] = str(msg.get(221),"utf

			if 360 in msg:
				msg_dict["NoAllocs"][0].append("EncodedAllocTextLen")
				for i in range(int(msg.get(78))):
					msg_dict["NoAllocs"][i+1].append(str(msg.get(360,i+1),"utf-8"))
			if 361 in msg:
				msg_dict["NoAllocs"][0].append("EncodedAllocText")
				for i in range(int(msg.get(78))):
					msg_dict["NoAllocs"][i+1].append(str(msg.get(361,i+1),"utf-8"))
			if 153 in msg:
				msg_dict["NoAllocs"][0].append("AllocAvgPx")
				for i in range(int(msg.get(78))):
					msg_dict["NoAllocs"][i+1].append(str(msg.get(153,i+1),"utf-8"))
			if 154 in msg:
				msg_dict["NoAllocs"][0].append("AllocNetMoney")
				for i in range(int(msg.get(78))):
					msg_dict["NoAllocs"][i+1].append(str(msg.get(154,i+1),"utf-8"))
			if 119 in msg:
				msg_dict["NoAllocs"][0].append("SettlCurrAmt")
				for i in range(int(msg.get(78))):
					msg_dict["NoAllocs"][i+1].append(str(msg.get(119,i+1),"utf-8"))
			if 737 in msg:
				msg_dict["NoAllocs"][0].append("AllocSettlCurrAmt")
				for i in range(int(msg.get(78)))

				msg_dict["LegCouponRate"] = str(msg.get(615),"utf-8")
			if 616 in msg:
				msg_dict["LegSecurityExchange"] = str(msg.get(616),"utf-8")
			if 617 in msg:
				msg_dict["LegIssuer"] = str(msg.get(617),"utf-8")
			if 618 in msg:
				msg_dict["EncodedLegIssuerLen"] = str(msg.get(618),"utf-8")
			if 619 in msg:
				msg_dict["EncodedLegIssuer"] = str(msg.get(619),"utf-8")
			if 620 in msg:
				msg_dict["LegSecurityDesc"] = str(msg.get(620),"utf-8")
			if 621 in msg:
				msg_dict["EncodedLegSecurityDescLen"] = str(msg.get(621),"utf-8")
			if 622 in msg:
				msg_dict["EncodedLegSecurityDesc"] = str(msg.get(622),"utf-8")
			if 623 in msg:
				msg_dict["LegRatioQty"] = str(msg.get(623),"utf-8")
			if 624 in msg:
				msg_dict["LegSide"] = str(msg.get(624),"utf-8")
			if 556 in msg:
				msg_dict["LegCurrency"] = str(msg.get(556),"utf-8")
			if 740 in msg:
				msg_dict["LegPool"] = str(msg.get(740),"utf-8")
			if 739 in msg:
				msg_dict["LegDatedDate"] = str(msg.get(739),"utf-8")
			if 955 in 

...

		if 647 in msg:
			msg_dict["MinBidSize"] = str(msg.get(647),"utf-8")
		if 134 in msg:
			msg_dict["BidSize"] = str(msg.get(134),"utf-8")
		if 648 in msg:
			msg_dict["MinOfferSize"] = str(msg.get(648),"utf-8")
		if 135 in msg:
			msg_dict["OfferSize"] = str(msg.get(135),"utf-8")
		if 62 in msg:
			msg_dict["ValidUntilTime"] = str(msg.get(62),"utf-8")
		if 188 in msg:
			msg_dict["BidSpotRate"] = str(msg.get(188),"utf-8")
		if 190 in msg:
			msg_dict["OfferSpotRate"] = str(msg.get(190),"utf-8")
		if 189 in msg:
			msg_dict["BidForwardPoints"] = str(msg.get(189),"utf-8")
		if 191 in msg:
			msg_dict["OfferForwardPoints"] = str(msg.get(191),"utf-8")
		if 631 in msg:
			msg_dict["MidPx"] = str(msg.get(631),"utf-8")
		if 632 in msg:
			msg_dict["BidYield"] = str(msg.get(632),"utf-8")
		if 633 in msg:
			msg_dict["MidYield"] = str(msg.get(633),"utf-8")
		if 634 in msg:
			msg_dict["OfferYield"] = str(msg.get(634),"utf-8")
		if 60 in msg:
			msg_dict["TransactTime"] = str(msg.get(60),"utf-8

			msg_dict["EncodedIssuerLen"] = str(msg.get(348),"utf-8")
		if 349 in msg:
			msg_dict["EncodedIssuer"] = str(msg.get(349),"utf-8")
		if 107 in msg:
			msg_dict["SecurityDesc"] = str(msg.get(107),"utf-8")
		if 350 in msg:
			msg_dict["EncodedSecurityDescLen"] = str(msg.get(350),"utf-8")
		if 351 in msg:
			msg_dict["EncodedSecurityDesc"] = str(msg.get(351),"utf-8")
		if 691 in msg:
			msg_dict["Pool"] = str(msg.get(691),"utf-8")
		if 667 in msg:
			msg_dict["ContractSettlMonth"] = str(msg.get(667),"utf-8")
		if 875 in msg:
			msg_dict["CPProgram"] = str(msg.get(875),"utf-8")
		if 876 in msg:
			msg_dict["CPRegType"] = str(msg.get(876),"utf-8")
		if 873 in msg:
			msg_dict["DatedDate"] = str(msg.get(873),"utf-8")
		if 874 in msg:
			msg_dict["InterestAccrualDate"] = str(msg.get(874),"utf-8")
		if 454 in msg:
			msg_dict["NoSecurityAltID"] = [[]]
			for i in range(int(msg.get(454))):
					msg_dict["NoSecurityAltID"].append([])
			if 455 in msg:
				msg_dict["NoSecurityAltID"][0].append

						msg_dict["NoUnderlyingStips"].append([])
				if 888 in msg:
					msg_dict["NoUnderlyingStips"][0].append("UnderlyingStipType")
					for i in range(int(msg.get(887))):
						msg_dict["NoUnderlyingStips"][i+1].append(str(msg.get(888,i+1),"utf-8"))
				if 889 in msg:
					msg_dict["NoUnderlyingStips"][0].append("UnderlyingStipValue")
					for i in range(int(msg.get(887))):
						msg_dict["NoUnderlyingStips"][i+1].append(str(msg.get(889,i+1),"utf-8"))
		if 702 in msg:
			msg_dict["NoPositions"] = [[]]
			for i in range(int(msg.get(702))):
					msg_dict["NoPositions"].append([])
			if 703 in msg:
				msg_dict["NoPositions"][0].append("PosType")
				for i in range(int(msg.get(702))):
					msg_dict["NoPositions"][i+1].append(str(msg.get(703,i+1),"utf-8"))
			if 704 in msg:
				msg_dict["NoPositions"][0].append("LongQty")
				for i in range(int(msg.get(702))):
					msg_dict["NoPositions"][i+1].append(str(msg.get(704,i+1),"utf-8"))
			if 705 in msg:
				msg_dict["NoPositions"][0].append("

		if 348 in msg:
			msg_dict["EncodedIssuerLen"] = str(msg.get(348),"utf-8")
		if 349 in msg:
			msg_dict["EncodedIssuer"] = str(msg.get(349),"utf-8")
		if 107 in msg:
			msg_dict["SecurityDesc"] = str(msg.get(107),"utf-8")
		if 350 in msg:
			msg_dict["EncodedSecurityDescLen"] = str(msg.get(350),"utf-8")
		if 351 in msg:
			msg_dict["EncodedSecurityDesc"] = str(msg.get(351),"utf-8")
		if 691 in msg:
			msg_dict["Pool"] = str(msg.get(691),"utf-8")
		if 667 in msg:
			msg_dict["ContractSettlMonth"] = str(msg.get(667),"utf-8")
		if 875 in msg:
			msg_dict["CPProgram"] = str(msg.get(875),"utf-8")
		if 876 in msg:
			msg_dict["CPRegType"] = str(msg.get(876),"utf-8")
		if 873 in msg:
			msg_dict["DatedDate"] = str(msg.get(873),"utf-8")
		if 874 in msg:
			msg_dict["InterestAccrualDate"] = str(msg.get(874),"utf-8")
		if 454 in msg:
			msg_dict["NoSecurityAltID"] = [[]]
			for i in range(int(msg.get(454))):
					msg_dict["NoSecurityAltID"].append([])
			if 455 in msg:
				msg_dict["NoSecurity

		if 854 in msg:
			msg_dict["QtyType"] = str(msg.get(854),"utf-8")
		if 15 in msg:
			msg_dict["Currency"] = str(msg.get(15),"utf-8")
		if 899 in msg:
			msg_dict["MarginExcess"] = str(msg.get(899),"utf-8")
		if 900 in msg:
			msg_dict["TotalNetValue"] = str(msg.get(900),"utf-8")
		if 901 in msg:
			msg_dict["CashOutstanding"] = str(msg.get(901),"utf-8")
		if 54 in msg:
			msg_dict["Side"] = str(msg.get(54),"utf-8")
		if 44 in msg:
			msg_dict["Price"] = str(msg.get(44),"utf-8")
		if 423 in msg:
			msg_dict["PriceType"] = str(msg.get(423),"utf-8")
		if 159 in msg:
			msg_dict["AccruedInterestAmt"] = str(msg.get(159),"utf-8")
		if 920 in msg:
			msg_dict["EndAccruedInterestAmt"] = str(msg.get(920),"utf-8")
		if 921 in msg:
			msg_dict["StartCash"] = str(msg.get(921),"utf-8")
		if 922 in msg:
			msg_dict["EndCash"] = str(msg.get(922),"utf-8")
		if 336 in msg:
			msg_dict["TradingSessionID"] = str(msg.get(336),"utf-8")
		if 625 in msg:
			msg_dict["TradingSessionSubID"] = str(msg.get(625